In [ ]:
# Get rid of annoying tf warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pickle
import numpy as np
np.set_printoptions(suppress=True)

import sys
sys.path.append("../")
import bayesflow as beef
from experiment import ModelComparisonExperiment

In [ ]:
experiment = ModelComparisonExperiment()

# Training

In [ ]:
with open('../data/training_data.pkl', 'rb') as f:
    training_data = pickle.load(f)
with open('../data/validation_data.pkl', 'rb') as f:
    validation_data = pickle.load(f)

In [ ]:
history = experiment.run(
    training_data=training_data,
    validation_data=validation_data
    )

In [ ]:
f = beef.diagnostics.plot_losses(history)